<a href="https://colab.research.google.com/github/aditya161205/DAV/blob/main/q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [31]:
train_data= pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')

train_data.head(1)

,Unnamed: 0,Gender,Age,Ever_Married,Family_Size,Profession,Graduated,Work_Experience,Energy_Consumption,Preferred_Renewable,Group
0,0,Male,22,No,4.0,Healthcare,No,1.0,Low,Hydro,D


In [21]:
train_data = train_data.drop('Unnamed: 0', axis=1)
test_data = test_data.drop('Unnamed: 0', axis=1)
train_data.head()
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Gender               8068 non-null   object 
 1   Age                  8068 non-null   int64  
 2   Ever_Married         7928 non-null   object 
 3   Family_Size          7733 non-null   float64
 4   Profession           7944 non-null   object 
 5   Graduated            7990 non-null   object 
 6   Work_Experience      7239 non-null   float64
 7   Energy_Consumption   8068 non-null   object 
 8   Preferred_Renewable  7992 non-null   object 
 9   Group                8068 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 630.4+ KB


In [22]:

numeric = train_data.select_dtypes(include=['float64']).columns
for col in numeric:
    train_data[col] = train_data[col].fillna(train_data[col].median())


cat = train_data.select_dtypes(include=['object']).columns
for col in cat:
    train_data[col] = train_data[col].fillna(train_data[col].mode()[0])


for col in train_data.columns:
    if train_data[col].dtype == 'float64':
        lower = train_data[col].quantile(0.25)
        upper = train_data[col].quantile(0.75)
        train_data[col] = train_data[col].clip(lower, upper)


train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Gender               8068 non-null   object 
 1   Age                  8068 non-null   int64  
 2   Ever_Married         8068 non-null   object 
 3   Family_Size          8068 non-null   float64
 4   Profession           8068 non-null   object 
 5   Graduated            8068 non-null   object 
 6   Work_Experience      8068 non-null   float64
 7   Energy_Consumption   8068 non-null   object 
 8   Preferred_Renewable  8068 non-null   object 
 9   Group                8068 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 630.4+ KB


In [29]:
numeric = test_data.select_dtypes(include=['float64']).columns
for col in numeric:
    test_data[col] = test_data[col].fillna(test_data[col].median())

cat = test_data.select_dtypes(include=['object']).columns
for col in cat:
    test_data[col] = test_data[col].fillna(test_data[col].mode()[0])

for col in test_data.columns:
    if test_data[col].dtype == 'float64':
        lower = test_data[col].quantile(0.25)
        upper = test_data[col].quantile(0.75)
        test_data[col] = test_data[col].clip(lower, upper)

test_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627 entries, 0 to 2626
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Gender               2627 non-null   object 
 1   Age                  2627 non-null   int64  
 2   Ever_Married         2627 non-null   object 
 3   Family_Size          2627 non-null   float64
 4   Profession           2627 non-null   object 
 5   Graduated            2627 non-null   object 
 6   Work_Experience      2627 non-null   float64
 7   Energy_Consumption   2627 non-null   object 
 8   Preferred_Renewable  2627 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 184.8+ KB


In [36]:

train_data_encoded=pd.get_dummies(train_data,columns = ['Gender', 'Ever_Married','Profession','Graduated','Energy_Consumption','Preferred_Renewable'], dtype = int, drop_first=True)

X = train_data_encoded.drop(['Group'], axis=1)
y=train_data_encoded['Group']


test_data_encoded=pd.get_dummies(test_data,columns = ['Gender', 'Ever_Married','Profession','Graduated','Energy_Consumption','Preferred_Renewable'], dtype = int, drop_first=True)

In [57]:

from sklearn.tree import DecisionTreeClassifier

from collections import Counter



In [75]:


class RandomForestClassifier:
    def __init__(self, n_estimators: int = 10, max_depth: int = 5, min_samples_leaf: int = 3, max_features: int | None = None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.trees: list = []

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        self.trees = []
        n_samples, n_features = X.shape
        self.max_features = self.max_features or int(np.sqrt(n_features))
        for _ in range(self.n_estimators):
            sample_index = np.random.choice(n_samples, size=n_samples, replace=True)
            X_sample, y_sample = X[sample_index], y[sample_index]

            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf)

            tree_features = np.random.choice(n_features, size=self.max_features, replace=False)
            X_sample_subfeatures = X_sample[:, tree_features]
            tree.fit(X_sample_subfeatures, y_sample)
            self.trees.append((tree, tree_features))

    def predict(self, X: np.ndarray) -> np.ndarray:

        all_preds = []
        for tree, features in self.trees:

            preds = tree.predict(X[:, features])
            all_preds.append(preds)


        all_preds = np.array(all_preds)

        def column_mode(arr: np.ndarray) -> list:

            return [Counter(col).most_common(1)[0][0] for col in arr.T]

        final_preds = np.array(column_mode(all_preds))
        return final_preds

In [76]:
model = RandomForestClassifier(n_estimators=30, max_depth=5, min_samples_leaf=3, max_features=2)



In [77]:

model.fit(X, y)

y_pred = model.predict(test_data_encoded)

print(y_pred)

['D' 'D' 'D' ... 'D' 'D' 'D']
